## CNN для распознавания гистоновых меток

In [ ]:
import pandas as pd
import os
import numpy as np
import keras
import tensorflow as tf
import sys

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv1D, Flatten
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('df[1-3].csv', sep = '\t', header = None)
df = df.dropna()

In [ ]:
[[type(i) == str for i in df[1].values]]

In [ ]:
df

### One hot encoding

In [ ]:
enc = {  'A':[1,0,0,0], 
         'C':[0,1,0,0], 
         'G':[0,0,1,0], 
         'T':[0,0,0,1]}

In [ ]:
df_encoded = np.array([[enc[char] for char in i] for i in df[1].values])

In [ ]:
df_encoded.shape

### Модель и обучающая выборка

In [ ]:
X = df_encoded

In [ ]:
y = (df[0] == 'yes').values.astype(np.int)

In [ ]:
X_train, X_ex, y_train, y_ex = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model = Sequential()
model.add(Conv1D(16, kernel_size = 6, input_shape = (200,4), activation = 'relu'))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=10, validation_split=0.3, epochs=100)

### Прогноз и статистики

In [ ]:
y_pred = model.predict(X_ex)

In [ ]:
y_pred = list([int(i) for i in y_pred])
y_ex = list([int(i) for i in y_ex])

In [ ]:
false_neg = 0; false_pos = 0; pos = 0; neg = 0
for i in range (0, len(y_pred)):
    if (y_ex[i] == 1) and (y_pred[i] != y_ex[i]): 
        false_neg+=1
    if (y_ex[i] == 0) and (y_pred[i] != y_ex[i]):
        false_pos+=1

In [ ]:
print(
'false positive rate ', false_pos/y_ex.count(0),
'\nfalse negative rate ', false_neg/y_ex.count(1),
'\nerrors ', false_neg + false_pos)

In [ ]:
true_neg= 0; true_pos = 0
for i in range (0, len(y_pred)):
    if (y_ex[i] == 1) and (y_pred[i] == y_ex[i]): 
        true_pos+=1
    if (y_ex[i] == 0) and (y_pred[i] == y_ex[i]):
        true_neg+=1
print('true negative ', true_neg, '\ntrue positive ', true_pos)
print('\nnumber of 0 ', y_ex.count(0), '\nnumber of 1 ', y_ex.count(1))